# Creating the data to test in STATA and Python

In [5]:
import warnings
warnings.filterwarnings("ignore") # I suppressed the warnings to make it easier to read, we should still try to fix them
import numpy as np
import pandas as pd
from ujive1 import *
from ujive2 import *
from tsls import * 

#Pick a vector length:
n = 1000

#Getting our Z's and making a Z matrix:
Z = np.random.randn(n, 1)
column_of_ones = np.ones((Z.shape[0], 1))
Z = np.hstack((column_of_ones, Z))
#pprint(Z)

#Parameter vectors:
α = np.array([1,1])
β = np.array([1,2])
#pprint(α)
#pprint(β)

#Error terms:
e1 = np.random.normal(0,5,n)
e2 = np.random.normal(0,5,n)
δ = np.random.normal(0,1)
ε = 5*e1 - 5*e2 + δ

#Making our endogenous variable:
x = np.dot(Z,α) + .2*e1
X = np.column_stack((column_of_ones, x))
#pprint(X)

#Outcome vector:
Y = np.dot(X,β) + ε

#OLS benchmark:
bhat_ols = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T, Y))

# Check to see if the Z'Z matrix is invertible
if np.linalg.matrix_rank(Z.T @ Z) == Z.shape[1]:  # Should be True
    print("Z'Z is invertible")
else:
    print("Z'Z is not invertible")

cond_number = np.linalg.cond(Z.T @ Z)
print(f"Condition number of Z.T @ Z: {cond_number}")
if cond_number > 1e10:  # Threshold for ill-conditioning
    raise ValueError("Z.T @ Z is ill-conditioned and may cause numerical instability.")


#2sls comparison:
Zt_Z = np.dot(Z.T, Z)
Zt_Z_inv = np.linalg.inv(Zt_Z)
pz = np.dot(np.dot(Z, Zt_Z_inv), Z.T)
proj_x = np.dot(pz, X)
first = np.linalg.inv(np.dot(proj_x.T, X))
second = np.dot(proj_x.T, Y)
bhat_2sls = np.dot(first, second)


ujive1 = UJIVE1(Y,X,Z,talk=False)
ujive2 = UJIVE2(Y,X,Z,talk=False)
tsls = TSLS(Y,X,Z, talk=False)

# Combine matrices into a single DataFrame
df = pd.DataFrame({
    "Y": Y,  # Outcome vector
    **{f"X{i}": X[:, i] for i in range(X.shape[1])},  # Endogenous variables
    **{f"Z{i}": Z[:, i] for i in range(Z.shape[1])}   # Instrumental variables
})

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

#Compare them:
print("OLS:", bhat_ols[1])
print("2SLS:", bhat_2sls[1])

Normally this estimator is used when Z has more columns than X. In this case Z has 2 columns and X has 2 columns.


Z'Z is invertible
Condition number of Z.T @ Z: 1.047894853021826
OLS: 14.594452077053553
2SLS: 1.5193036902029196


In [6]:
print(ujive1.summary())


UJIVE1 Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    1.854067    1.725258 1.074661 0.282786       -1.531482         5.239616
    1.461718    1.254936 1.164775 0.244388       -1.000898         3.924333
--------------------------------------------------------------------------------
R-squared: 0.058766
Adjusted R-squared: 0.057823
F-statistic: 3.284886
Root MSE: 34.945788
None


In [7]:
print(ujive2.summary())


UJIVE2 Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    1.859003    1.725034 1.077661 0.281445       -1.526108         5.244113
    1.456891    1.254328 1.161492 0.245720       -1.004531         3.918313
--------------------------------------------------------------------------------
R-squared: 0.058583
Adjusted R-squared: 0.057639
F-statistic: 3.262591
Root MSE: 34.949201
None


In [8]:
print(tsls.summary())


TSLS Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    0.897589    0.860360 1.043271 0.297076       -0.790736         2.585913
    0.897589    0.860360 1.043271 0.297076       -0.790736         2.585913
    1.519304    1.250453 1.215003 0.224653       -0.934518         3.973125
--------------------------------------------------------------------------------
R-squared: 0.060955
Adjusted R-squared: 0.059071
F-statistic: 1.776757
Root MSE: 34.922641
None


In [9]:
(Z @ np.linalg.inv(np.dot(Z.T,Z)))

array([[ 0.00100292, -0.00043674],
       [ 0.00100737, -0.00110145],
       [ 0.00101286, -0.0019215 ],
       ...,
       [ 0.00099508,  0.00073497],
       [ 0.00099735,  0.00039566],
       [ 0.0010059 , -0.00088094]], shape=(1000, 2))

In [10]:
(Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T

array([[ 0.00118239,  0.00145998,  0.00180245, ...,  0.00069307,
         0.00083477,  0.00136789],
       [ 0.00145998,  0.00216006,  0.00302374, ...,  0.00022593,
         0.00058329,  0.00192781],
       [ 0.00180245,  0.00302374,  0.00453046, ..., -0.00035039,
         0.00027303,  0.00261859],
       ...,
       [ 0.00069307,  0.00022593, -0.00035039, ...,  0.00151652,
         0.00127806,  0.0003809 ],
       [ 0.00083477,  0.00058329,  0.00027303, ...,  0.00127806,
         0.00114969,  0.00066671],
       [ 0.00136789,  0.00192781,  0.00261859, ...,  0.0003809 ,
         0.00066671,  0.00174206]], shape=(1000, 1000))